In [1]:
import os, sys, json, glob
import numpy as np
import matplotlib.pyplot as plt
import girder_client
import histomicstk.utils as htk_utils
from cStringIO import StringIO
from IPython.display import Image as IPImage  
import histomicstk.utils as htk_utils
import io, random
from PIL import Image, ImageDraw
import logging

logging.getLogger("requests").setLevel(logging.WARNING)

%matplotlib inline

INFO:girder:Created LRU Cache for 'tilesource' with 104 maximum size
INFO:root:Notice: Could not import MapnikTileSource


In [2]:
gc = girder_client.GirderClient(apiUrl="http://candygram.neurology.emory.edu:8080/api/v1")
gc.authenticate(interactive=True)

Login or email: admin
Password for admin: ········


In [3]:
# Fetches all Cohort slides
def getSlidesInCohort(Cohortname, nSlides):
    tcgaCohorts = gc.get('/tcga/cohort')   
    cohortInfo = dict([(x['name'],x['_id']) for x in tcgaCohorts['data']])
    slidesInCohort = gc.get('/tcga/cohort/%s/images?limit=%d' % (cohortInfo[Cohortname],nSlides))
    return slidesInCohort

In [4]:
# Fetches DX1 cohort slides only
def getDXSlidesInCohort(Cohortname, nSlides):
    totalDX1SlidesFound = 0
    dxSlidesForCohort = []
    tcgaCohorts = gc.get('/tcga/cohort')   
    cohortInfo = dict([(x['name'],x['_id']) for x in tcgaCohorts['data']])
    slidesInCohort = gc.get('/tcga/cohort/%s/images?limit=%d' % (cohortInfo[Cohortname],nSlides))
    for sld in slidesInCohort['data']:
        if 'DX1' in sld['tcga']['barcode']:
            totalDX1SlidesFound +=1
            dxSlidesForCohort.append(sld)
        else:
            pass
    return dxSlidesForCohort

In [5]:
# Code which masks the non-white spaces and extract X,Y coordiantes
def grabTiles(sID, sampleSlide):
    sID = sampleSlide['_id']
    lowResMag = 1.25
    lowResImg = gc.get('/item/%s/tiles/region?magnification=%s' % ( sID, lowResMag),jsonResp=False)
    lowResPILimage = Image.open(io.BytesIO(lowResImg.content))
    im_fgnd_mask_lres = htk_utils.simple_mask(np.asarray(lowResPILimage))
    (YmaskPts,XmaskPts)  = np.nonzero(im_fgnd_mask_lres)
    maskCoords = zip(YmaskPts,XmaskPts)
    return lowResImg, maskCoords    

In [6]:
# Tiles save function - to save tiles into a file
def tileSave(sID,nTiles,lowResImg, slideBaseName, out_path, maskCoords):
    sldBaseName = slideBaseName
    lowResMag = 1.25
    mask_Coords=maskCoords
    outpath = out_path
    num_tiles=nTiles
    
    random.shuffle(mask_Coords)  
    outputRes = 20
    scaleFactor = outputRes/lowResMag
    
    maxx = len(mask_Coords)
    tilecount=0
    for idx, c in enumerate(mask_Coords):
        print("Image %s of %s" % (idx, maxx))
        curTile = gc.get('/item/%s/tiles/region?magnification=%s&top=%d&left=%d&regionWidth=%d&regionHeight=%d' 
                       % ( sID, outputRes, c[0]*scaleFactor, c[1]*scaleFactor,256,256),jsonResp=False)
        img = Image.open(io.BytesIO(curTile.content))
        avg = np.average(img)
        if avg > 150 and avg < 210:
            tilename = sldBaseName + '_%dx_%d_%d.png' % (outputRes, c[0], c[1]) 
            img.save(outpath + tilename)
            tilecount+=1
        
        if tilecount == num_tiles:
            break

In [7]:
# Function to fetch tiles randomly from each slide in given Cohort set
def randomTileSave(dxSlideCohort,cancerClass, train_or_testpath, tileCount):
    cancerClass= cancerClass
    outpath=train_or_testpath
    import random
    for slideNumber in range(100):
        slideNumber = random.randint(1,101)
        sampleSlide = dxSlideCohort['data'][slideNumber]
        sID = sampleSlide['_id']
        lowResImg, maskCoords = grabTiles(sID, sampleSlide)
        slideBaseName = sampleSlide['name'].split(".")[0]
        out_path = '/data/'+outpath+'/'+cancerClass+'/'
        nTiles=tileCount
        tileSave(sID, nTiles, lowResImg, slideBaseName, out_path, maskCoords)  

In [ ]:
# Fetches, 5 tiles each from  100 gbm slides for training
cancerClass="gbm"
storPath="train"
dxSlideCohort = getSlidesInCohort('gbm',100)
randomTileSave(dxSlideCohort,cancerClass,storPath,5) 

Image 0 of 707867
Image 1 of 707867
Image 2 of 707867
Image 3 of 707867
Image 4 of 707867
Image 5 of 707867
Image 6 of 707867
Image 7 of 707867
Image 8 of 707867
Image 9 of 707867
Image 10 of 707867
Image 11 of 707867
Image 12 of 707867
Image 13 of 707867
Image 14 of 707867
Image 15 of 707867
Image 16 of 707867
Image 17 of 707867
Image 18 of 707867
Image 19 of 707867
Image 20 of 707867
Image 21 of 707867
Image 22 of 707867
Image 23 of 707867
Image 24 of 707867
Image 25 of 707867
Image 26 of 707867
Image 27 of 707867
Image 28 of 707867
Image 29 of 707867
Image 30 of 707867
Image 31 of 707867
Image 32 of 707867
Image 33 of 707867
Image 34 of 707867
Image 35 of 707867
Image 36 of 707867
Image 37 of 707867
Image 38 of 707867
Image 39 of 707867
Image 40 of 707867
Image 41 of 707867
Image 42 of 707867
Image 43 of 707867
Image 44 of 707867
Image 45 of 707867
Image 46 of 707867
Image 47 of 707867
Image 48 of 707867
Image 49 of 707867
Image 50 of 707867
Image 51 of 707867
Image 52 of 707867
Ima

Image 13 of 1740856
Image 14 of 1740856
Image 15 of 1740856
Image 16 of 1740856
Image 17 of 1740856
Image 18 of 1740856
Image 19 of 1740856
Image 0 of 349745
Image 1 of 349745
Image 2 of 349745
Image 3 of 349745
Image 4 of 349745
Image 5 of 349745
Image 6 of 349745
Image 7 of 349745
Image 8 of 349745
Image 9 of 349745
Image 10 of 349745
Image 11 of 349745
Image 12 of 349745
Image 13 of 349745
Image 14 of 349745
Image 15 of 349745
Image 16 of 349745
Image 17 of 349745
Image 18 of 349745
Image 19 of 349745
Image 20 of 349745
Image 21 of 349745
Image 22 of 349745
Image 23 of 349745
Image 24 of 349745
Image 25 of 349745
Image 26 of 349745
Image 27 of 349745
Image 28 of 349745
Image 29 of 349745
Image 30 of 349745
Image 31 of 349745
Image 32 of 349745
Image 33 of 349745
Image 34 of 349745
Image 35 of 349745
Image 36 of 349745
Image 0 of 1547024
Image 1 of 1547024
Image 2 of 1547024
Image 3 of 1547024
Image 4 of 1547024
Image 5 of 1547024
Image 6 of 1547024
Image 0 of 4282322
Image 1 of 4282

Image 9 of 156567
Image 10 of 156567
Image 11 of 156567
Image 0 of 194010
Image 1 of 194010
Image 2 of 194010
Image 3 of 194010
Image 4 of 194010
Image 5 of 194010
Image 6 of 194010
Image 7 of 194010
Image 8 of 194010
Image 9 of 194010
Image 10 of 194010
Image 11 of 194010
Image 12 of 194010
Image 13 of 194010
Image 14 of 194010
Image 0 of 299691
Image 1 of 299691
Image 2 of 299691
Image 3 of 299691
Image 4 of 299691
Image 5 of 299691
Image 6 of 299691
Image 7 of 299691
Image 8 of 299691
Image 9 of 299691
Image 10 of 299691
Image 11 of 299691
Image 12 of 299691
Image 0 of 4509205
Image 1 of 4509205
Image 2 of 4509205
Image 3 of 4509205
Image 4 of 4509205
Image 0 of 2490889
Image 1 of 2490889
Image 2 of 2490889
Image 3 of 2490889
Image 4 of 2490889
Image 5 of 2490889
Image 6 of 2490889
Image 7 of 2490889
Image 8 of 2490889
Image 9 of 2490889
Image 10 of 2490889
Image 0 of 2486686
Image 1 of 2486686
Image 2 of 2486686
Image 3 of 2486686
Image 4 of 2486686
Image 5 of 2486686
Image 6 of 24

Image 5 of 1698437
Image 6 of 1698437
Image 7 of 1698437
Image 0 of 1429423
Image 1 of 1429423
Image 2 of 1429423
Image 3 of 1429423
Image 4 of 1429423
Image 5 of 1429423
Image 6 of 1429423
Image 7 of 1429423
Image 8 of 1429423
Image 9 of 1429423
Image 10 of 1429423
Image 11 of 1429423
Image 12 of 1429423
Image 13 of 1429423
Image 14 of 1429423
Image 15 of 1429423
Image 16 of 1429423
Image 17 of 1429423
Image 18 of 1429423
Image 19 of 1429423
Image 20 of 1429423
Image 21 of 1429423
Image 0 of 1770127
Image 1 of 1770127
Image 2 of 1770127
Image 3 of 1770127
Image 4 of 1770127
Image 5 of 1770127
Image 6 of 1770127
Image 0 of 1632127
Image 1 of 1632127
Image 2 of 1632127
Image 3 of 1632127
Image 4 of 1632127
Image 0 of 483436
Image 1 of 483436
Image 2 of 483436
Image 3 of 483436
Image 4 of 483436
Image 5 of 483436
Image 6 of 483436
Image 7 of 483436
Image 8 of 483436
Image 9 of 483436
Image 10 of 483436
Image 11 of 483436
Image 12 of 483436
Image 13 of 483436
Image 14 of 483436
Image 15 o

Image 3 of 942093
Image 4 of 942093
Image 5 of 942093
Image 6 of 942093
Image 0 of 1811956
Image 1 of 1811956
Image 2 of 1811956
Image 3 of 1811956
Image 4 of 1811956
Image 5 of 1811956
Image 6 of 1811956
Image 7 of 1811956
Image 8 of 1811956
Image 0 of 3845325
Image 1 of 3845325
Image 2 of 3845325
Image 3 of 3845325
Image 4 of 3845325
Image 5 of 3845325
Image 6 of 3845325
Image 0 of 943277
Image 1 of 943277
Image 2 of 943277
Image 3 of 943277
Image 4 of 943277
Image 5 of 943277
Image 6 of 943277
Image 7 of 943277
Image 0 of 4524352
Image 1 of 4524352
Image 2 of 4524352
Image 3 of 4524352
Image 4 of 4524352
Image 0 of 488411
Image 1 of 488411
Image 2 of 488411
Image 3 of 488411
Image 4 of 488411
Image 5 of 488411
Image 6 of 488411
Image 7 of 488411
Image 8 of 488411
Image 9 of 488411
Image 10 of 488411
Image 11 of 488411
Image 12 of 488411
Image 13 of 488411
Image 14 of 488411
Image 15 of 488411
Image 16 of 488411
Image 17 of 488411
Image 18 of 488411
Image 19 of 488411
Image 20 of 488

In [ ]:
# Fetches, 5 tiles each from  100 gbm slides for test
cancerClass="gbm"
storPath="test"
dxSlideCohort = getSlidesInCohort('gbm',100)
randomTileSave(dxSlideCohort,cancerClass,storPath,5) # Fetches, gbm class tiles for training

In [ ]:
# Fetches, 5 tiles each from  100 lgg slides for training
cancerClass="lgg"
storPath="train"
dxSlideCohort = getSlidesInCohort('lgg',100)
randomTileSave(dxSlideCohort,cancerClass,storPath,5) # Fetches, gbm class tiles for training

In [ ]:
# Fetches, 5 tiles each from  100 lgg slides for test
cancerClass="lgg"
storPath="test"
dxSlideCohort = getSlidesInCohort('lgg',100)
randomTileSave(dxSlideCohort,cancerClass,storPath,5) # Fetches, gbm class tiles for training

In [ ]:
## Generate the training and test Sets for all the cohorts... could potentially ignore cohorts if they
## Don't have enough images
combinedTrainSet = {}
combinedTestSet = {}
combinedValSet = {}

for c in cohortData:
    cohortSamples = cohortData[c]['dxCaseList']
    print c, len(cohortSamples)
    cohortTrain,cohortVal,cohortTest = generateTrainTestVal(cohortSamples)
    combinedTrainSet[c] = cohortTrain
    combinedTestSet[c]  = cohortTest
    combinedValSet[c] = cohortVal